In [1]:
%matplotlib notebook
%pylab
from math import*

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def legendre_poly(p,x):

    L1=0;L1_1=0;L1_2=0;
    L0=1;L0_1=0;L0_2=0;

    for i in range(1,p+1):
        L2=L1
        L2_1=L1_1
        L2_2=L1_2
        
        L1=L0
        L1_1=L0_1
        L1_2=L0_2
        
        a=(2*i-1)/i
        b=(i-1)/i
        L0=a*x*L1 - b*L2
        L0_1=a*(L1 + x*L1_1) - b*L2_1
        L0_2=a*(2*L1_1 + x*L1_2) - b*L2_2
    
    return L0, L0_1, L0_2
    

In [3]:
# The Legendre-Gauss-Lobatto points and weights
#which are the roots of the Lobatto Polynomials.

def legendre_gauss_lobatto(P):

    p=P-1;
    ph = int((p+1)/2);
    xgl = zeros(P+1)
    wgl = zeros(P+1)
    for i in range(1,ph+1):
        x=cos((2*i-1)*pi/(2*p+1))
        for k in range(1,20):
            L0,L0_1,L0_2 =legendre_poly(p,x)

            #Get new Newton Iteration
            dx=-(1-x**2)*L0_1/(-2*x*L0_1 + (1-x**2)*L0_2)
            x=x+dx
            if (abs(dx) < 1.0e-20):
                break

        xgl[p+2-i]=x
        wgl[p+2-i]=2/(p*(p+1)*L0**2)
        
    #Check for Zero Root
    if (p+1 != 2*ph):
        x=0;
        L0,L0_1,L0_2=legendre_poly(p,x)
        xgl[ph+1]=x;
        wgl[ph+1]=2/(p*(p+1)*L0**2)

    #Find remainder of roots via symmetry
    for i in range(1,ph+1):
        xgl[i]=-xgl[p+2-i]
        wgl[i]=+wgl[p+2-i]
    
    return array(xgl), array(wgl)

In [4]:
def LGL_grid_elemnts_dg(ngl, nelem, xgl):
    # constants
    xmin = -1
    xmax = +1
    dx = (xmax-xmin)/nelem
    # Grid points
    ip = 1
    coord = zeros((ngl, nelem))
    intma = zeros((ngl, nelem))
    coord[0,0] = xmin
    for i in range(1, nelem+1):
        x0 = xmin + (i-1)*dx
        cood[0,i-1] = x0
        intma[0,i-1] = ip
        for j in range(2, ngl+1):
            ip=ip + 1
            coord[j-1,i] = (xgl[j-1]+1)*(dx/2) + x0
            intma[j-1,i] = ip
            
    return coord, intma

In [5]:
def creat_Fmatrix_dg(inode,npoin,nelem,ngl,u,diss):

    Fmatrix=zeros(npoin,npoin)

    for e in range(1,nelem+1):

        #Left Side
        left=e-1
        if (e == 1):
            left=nelem
        
        i0=1
        iN=ngl
        n_left=-1

        IE=inode[i0,e]
        IL=inode[iN,left]

        Fmatrix[IE,IE]=n_left*u/2*(1+n_left*diss)
        Fmatrix[IE,IL]=n_left*u/2*(1-n_left*diss)

        #Right Side
        right=e+1
        if (e == nelem):
            right=1
        end
        i0=1
        iN=ngl
        n_right=1

        IE=inode[iN,e]
        IR=inode[i0,right]

        Fmatrix[IE,IE]=n_right*u/2*(1+n_right*diss)
        Fmatrix[IE,IR]=n_right*u/2*(1-n_right*diss)
    
    return Fmatrix



In [6]:

def create_mass_dg(coord,nelem,ngl,nq,wnq,psi):

    #Initialize
    mass = {}
    mass_e = zeros((ngl,ngl))
    
    for ie in range(1,nelem+1):
        
        #Store Coordinates
        for i in range(0,ngl):
            x[i]=coord[i,ie-1]

        dx=x[ngl]-x[1]
        jac=dx/2

        #Do LGL Integration
        for l in range(0, nq):
            wq=wnq[l]*jac
            for i in range(0,ngl):
                h_i=psi[i,l]
                for j in range(0,ngl):
                    h_j=psi[j,l]
                    mass_e[i,j]=mass_e[i,j] + wq*h_i*h_j
                    
        mass[ie] = mass_e
    
    return mass



In [7]:
def create_diff_matrix_dg(ngl,nq,wnq,psi,dpsi):

    #Initialize
    Dmatrix = zeros((ngl,ngl))

    #Integrate Divergence of Flux
    #for ie=1:nelem

       #LGL Integration
    for i in range(0,ngl):
        for j in range(0,ngl):
            for k in range(0,nq):
                wk=wnq(k)
                dhdx_ik=dpsi(i,k)
                h_jk=psi(j,k)
                Dmatrix[i,j]=Dmatrix[i,j] + wk*dhdx_ik*h_jk
    
    return Dmatrix

In [8]:
def create_Lmatrix_dg(coord,nelem,ngl,nq,wnq,dpsi):

    #Initialize
    lpla = zeros((ngl,ngl))
    laplacian_matrix = {}
    

    for e in range(1,nelem+1):
        x = zeros(ngl)
        #Store Coordinates
        for i in range(ngl):
            x[i]=coord[i,e-1]

        dx=x[ngl]-x[1]
        jac=dx/2
        dksi_dx=2/dx

        #LGL Integration
        for i in range(ngl):
            for j in range(ngl):
                for k in range(nq):
                    wq=wnq[k]*jac
                    dhdx_ik=dpsi[i,k]*dksi_dx
                    dhdx_jk=dpsi[j,k]*dksi_dx
                    lpla[i,j] = lpla[i,j] + wq*dhdx_ik*dhdx_jk
        
        laplacian_matrix[e] = lpla
        
    return laplacian_matrix

In [9]:
def lagrange_basis3(P,Q,xgl,xnq):

    #Get Quadrature roots
    #[xnq,wnq] = legendre_gauss_lobatto(Q);
    psi = zeros((P,Q))
    dpsi = zeros((P,Q))
    #Perform Quadrature
    for l in range(Q):
        xl=xnq[l]

        #Construct Basis
        for i in range(P):
            xi=xgl[i]
            psi[i,l]=1
            dpsi[i,l]=0
            for j in range(P):
                xj=xgl[j]
                #Basis
                if (j != i):
                    psi[i,l]=psi[i,l]*(xl-xj)/(xi-xj)
                    
                ddpsi=1
                if (j != i):
                    for k in range(P):
                        xk=xgl[k];
                        #Derivative of Basis
                        if (k !=i and k !=j):
                            ddpsi = ddpsi*(xl-xk)/(xi-xk)


                    dpsi[i,l]=dpsi[i,l] + ddpsi/(xi-xj)
                    
    return psi, dpsi
             


In [10]:
def apply_filter_dg(qp,f,nelem,ngl):

    #Initialize
    rhs = zeros((ngl,nelem))
    q_e = zeros(ngl)
    qf = zeros(ngl)

    #Integrate Divergence of Flux
    for ie in range(1,nelem+1):

        print("new iteration")
        print(ie)
        print(nelem)
        print(len(qp))
        print(len(q_e))

        #Store Local Solution
        for i in range(ngl):
            q_e[i]=qp[i,ie-1]

        #Form Filtered Variable
        qf=f*q_e

        #Store It
        for i in range(ngl):
            rhs[i,ie-1] = qf[i]
    
    return rhs

In [11]:
def exact_solution_dg(coord,nelem,ngl,time,icase):

    #Set some constants
    w=1
    h=0
    xc=0
    xmin=-1
    xmax=+1
    xl=xmax-xmin
    sigma0=0.125
    rc=0.125
    sigma=sigma0
    u=w*xl
    #icase = 0

    #Initialize
    qe=zeros((ngl,nelem))

    timec=time - floor(time)

    #Generate Grid Points
    for ie in range(1,nelem+1):
        for i in range(ngl):
            x=coord[i,ie-1]
            xbar=xc + u*timec
            if (xbar > xmax): 
                xbar=xmin + (xbar-xmax)
          
            r = x-xbar
            if (icase == 1):
                #qe(i,ie)=sigma0/sigma*exp( -(x-xbar)^2/(4*sigma^2) );
                qe[i,ie-1]=exp(-128*(x-xbar)**2)
                #qe(i,ie)=sigma0/sigma*exp( -(x-xbar)^2/(sigma^2) );
            elif (icase == 2):
                 if (abs(r) <= rc):
                    qe[i,ie-1]=1
                
            elif (icase == 3):
                qe[i,ie]=sigma0/sigma*exp(-(x-xbar)**2/(2*sigma**2));
            elif (icase == 4):
                if (abs(r) <= rc):
                    qe[i,ie-1]=1
                    
            elif (icase == 5):
                if (x <= xc):
                    qe[i,ie-1]=1

            elif (icase == 6):
                qe[i,ie-1] = sin(2*pi*x)

    return qe

In [12]:
def source_function_dg(coord,nelem,ngl,time,icase):

    #Set some constants
    w=1;
    h=0;
    xmin=-1;
    xmax=+1;
    xl=xmax-xmin;
    sigma0=0.125;
    xc=-0.7;
    #xc=-0.75;
    rc=1.e-6;
    #rc=0.125;
    fc=10;
    sigma = sigma0
    u=w*xl;
    #icase;

    #Initialize
    fe=zeros((ngl,nelem))

    timec=time - floor(time)

    #Generate Grid Points
    for ie in range(1,nelem+1):
        for i in range(ngl):
            x = coord[i,ie-1]
            r = abs(x-xc)
            if (icase <= 2): 
                fe[i,ie-1]=0
            elif (icase == 3):
                #fe(i,ie)=sigma0/sigma*exp( -(x-xc)^2/(2*sigma^2) );
                if (r <= rc): 
                    fe[i,ie-1] =fc

            elif (icase == 4):
                #fe(i,ie)=sigma0/sigma*exp( -(x-xc)^2/(2*sigma^2) );
                if (r <= rc): 
                    fe[i,ie-1] =fc
                    
            elif (icase == 5): 
                fe[i,ie-1] = 0
            elif (icase == 6): 
                fe[i,ie-1]=0
    

In [13]:
def filter_init(P,xgl,xmu):

    #Constants
    p=P-1
    ph=floor((p+1)/2)
    alpha=17
    order=18

    #Initialize
    leg=zeros((P,P))
    f=zeros((P,P))

    #Compute Legendre Polynomial Matrix
    for i in range(P):
        x=xgl[i]
        for j in range(P):
            jj=j
            L0,L0_1,L0_2=legendre_poly(jj,x);
            leg[i,j]=L0;

    leg_inv=linalg.inv(leg)

    #Compute Weight
    weight = zeros(P)
    for i in range(P):
        weight[i]=exp(-alpha*((i-1)/p)**order)

    ibeg=P-4;
    iend=P;
    idelta=iend-ibeg;
    idelta=0.05/idelta;
    for i in range(ibeg-1):
        weight[i]=1;

    for i in range(ibeg,iend+1):
        weight[i]=1.0 - (i-ibeg)*idelta;

    #weight;

    #ibeg=round(2/3*P);
    ibeg=1;
    iend=P;
    for i in range(ibeg):
        weight[i]=1;

    for i in range(ibeg+1,iend): # verification
        weight[i]=1.0 - ((i-ibeg)/(iend-ibeg))**2;

    #weight;

    #Construct 1D Filter Matrix
    for i in range(P):
        for j in range(P):
            sum=0;
            for k in range(P):
                sum=sum + leg[i,k]*weight(k)*leg_inv[k,j]

            f[i,j]=xmu*sum;

        f[i,i]=f[i,i] + (1-xmu);

    return f

In [15]:

#Input Data
#nelem=3; %Number of Elements == 22
#nop1=2;
#nop2=2;    %Interpolation Order == 5

nop_array = [2,4,6,8];
nelem_array = [20,40,80,160];

kstages=3 #RK2, RK3, RK34
dt=1e-2  #time-step, fraction of one revolution
Courant_max=0.05
time_final=0.025  #final time in revolutions
nplots=50 #plotting variable - Ignore
iplot_solution=0 #Switch to Plot or Not.
interpolation_points=1  # %=1 for LGL; =2 equi-spaced points; =3 for LG
integration_points=1 # %=1 for LGL and =2 for LG
integration_type=2 # %=1 is inexact and =2 is exact
space_method_type='dg'# %=1 for CG and =2 for DG

icase=6  # case number: 1 is a Gaussian, 2 is a square wave, 3 is
         #Gaussian with source and 4 is square wave with source.
         
#stabilization - won't need it for now         
xmu=0.0 # filtering strength: 1 is full strength and 0 is no filter
ifilter=10000  #time-step frequency that the filter is applied.
diss=0  
#visc=0.000 #for CG N=4 and N=5
nlaplacian=0

#if (nlaplacian == 0) 
#    visc=0;
#end

#Store Constants
# ngl=nop + 1;
ntime=time_final/dt;
l2_norm = zeros((len(nop_array), len(nelem_array)))
npoin_array = zeros((len(nop_array), len(nelem_array)))

for i_nop in range(len(nop_array)):
    nop = nop_array[i_nop]
    ngl=nop+1
    
    for i_nelem in range(len(nelem_array)):
        nelem = nelem_array[i_nelem]
        npoin=ngl*nelem;
        
        npoin_array[i_nop,i_nelem] = npoin
        #Compute Interpolation and Integration Points
        
        xgl,wgl=legendre_gauss_lobatto(ngl)
        #method_text='_Interp=LGL'
        
        if (integration_points == 1):
            integration_text=['LGL']
            if (integration_type == 1):
                noq=nop;
            elif (integration_type == 2):
                noq=nop+1
            
            nq=noq + 1
            xnq,wnq =legendre_gauss_lobatto(nq)

        #Compute Lagrange Polynomial and derivatives
        psi,dpsi = lagrange_basis3(ngl,nq,xgl,xnq)

        #Compute Filter Matrix
        f = filter_init(ngl,xgl,xmu)

        #Create Grid
        coord,intma = create_grid_dg(ngl,nelem,xgl)
        dx = coord(2,1)-coord(1,1)
        u=2;
        dt=Courant_max*dx/u
        ntime=round(time_final/dt)
        dt=time_final/ntime
        Courant=u*dt/dx

        #Compute Exact Solution
        time=0
        qe = exact_solution_dg(coord,nelem,ngl,time,icase)
        fe = source_function_dg(coord,nelem,ngl,time,icase)

        #Create Local/Element Mass and Differentiation Matrices
        mass = create_mass_dg(coord,nelem,ngl,nq,wnq,psi)
        diff_matrix = create_diff_matrix_dg(ngl,nq,wnq,psi,dpsi)
        laplacian_matrix = create_Lmatrix_dg(coord,nelem,ngl,nq,wnq,dpsi)

        #Form Global Matrix and Periodic BC Pointers
        inode=zeros((ngl,nelem))
        iperiodic=empty(npoin)
        
        if space_method_type == 'cg':
            inode=intma;
            for i in range(npoin):
                iperiodic[i]=i
            iperiodic[npoin]=iperiodic[0]
        elif space_method_type == 'dg':
            ip=0
            for e in range(1,nelem+1):
                for i in range(ngl):
                    ip=ip+1;
                    inode[i,e]=ip0
            
            for i in range(npoin):
                iperiodic[i]=i;

        #Form Global Mass and Differentiation Matrices
        Mmatrix=zeros((npoin,npoin))
        Dmatrix=zeros((npoin,npoin))
        Fmatrix=zeros((npoin,npoin))
        Lmatrix=zeros((npoin,npoin))
        for e in range(1,nelem):
            for i in range(ngl):
                ip=iperiodic(inode[i,e])
                for j in range(ngl):
                    jp=iperiodic[inode[j,e]]
                    Mmatrix[ip,jp]=Mmatrix[ip,jp] + mass[e][i,j]
                    Dmatrix[ip,jp]=Dmatrix[ip,jp] + u*diff_matrix[i,j]
                    Lmatrix[ip,jp]=Lmatrix[ip,jp] + laplacian_matrix[e][i,j]
        
        if space_method_type == 'cg':
            Mmatrix[npoin,npoin]=1
        elif space_method_type == 'dg':
            Fmatrix = create_Fmatrix_dg(inode,npoin,nelem,ngl,u,diss)
            
        Lmatrix_hat= solve(Mmatrix,Lmatrix)

        #Construct Full RHS matrix
        if (nlaplacian > 0):
            HV_matrix = eye(npoin)
        elif (nlaplacian == 0):
            HV_matrix=zeros(npoin)
        
        for i in range(nlaplacian):
            HV_matrix=Lmatrix_hat*HV_matrix
        
        Rmatrix= solve(Mmatrix,(Dmatrix - Fmatrix))

        #Left-Multiply by Inverse Mass Matrix
        Dmatrix_hat=Rmatrix

        #Form Long Exact Solution Vector
        qexact=zeros(npoin)
        for e in range(1,nelem):
            for i in range(ngl):
                ip=inode(i,e)
                qexact[ip]=qe[i,e]

        #Initialize State Vector
        qexact= transpose(qexact)
        q1=qexact
        q0=qexact
        qp=qexact
        iplot=round(ntime/nplots)

        #Time Integration
        for itime in range(ntime):
            time=time + dt;

            #disp(['itime =  ',num2str(itime),' time = ', num2str(time),' courant = ', num2str(Courant)]);
            for ik in range(1,kstages):
                
                if(ik == 1):
                    
                    a0=1
                    a1=0
                    beta=1.0/2.0
                        
                if(ik == 2):
                    a0=0
                    a1=1
                    beta=1.0/2.0
                       
                if(ik == 3):
                    a0=2.0/3.0
                    a1=1.0/3.0
                    beta=1.0/6.0
                      
                if (ik == 4):
                    a0=0
                    a1=1
                    beta=1.0/2.0
                        
                dtt=dt*beta;
                qp=a0*q0 + a1*q1 + dtt*Dmatrix_hat*qp;

                #apply Periodic bc
                if space_method_type == 'cg':
                    qp[npoin]=qp[periodic[npoin]] 
              
                #Update
                q1=qp;

            #Filter Solution
            if (mod(itime,ifilter) == 0):
                rhs = apply_filter_dg(qp,f,nelem,ngl)
                qp=rhs

            #Update Q
            q0=qp;
        

        #Compute Exact Solution
        qe = exact_solution_dg(coord,nelem,ngl,time,icase)
        #Form Long Exact Solution Vector
        for e in range(1,nelem):
            for i in range(ngl):
                ip=inode[i,e]
                qexact[ip]=qe[i,e]
        #Compute Norm
        top=0
        bot=0
        error=zeros(npoin)
        for i in range(npoin):
            top=top + (q0[i]-qexact[i])**2
            error[i]=(q0[i]-qexact[i])
            bot=bot + qexact[i]**2
        
        l2_norm[i_nop,i_nelem] = sqrt(top/bot)

        #Compute a gridpoint solution
        x_sol=zeros(npoin,1);
        for ie in range(1,nelem+1):
            for i in range(ngl):
                ip= int(inode[i,ie-1])
                x_sol[ip]=coord[i,ie-1]



/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars
/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars
/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars
/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


LinAlgError: Singular matrix